In [ ]:
import builtins
import copy

from tqdm import tqdm
from pytype.main import _run_pytype
from pytype import config
from ast import Name

import crawl.run

options = config.Options.create("temp.py")

In [ ]:
import ast

with open("temp.py", "r") as file:
    source = file.read()
    original_tree = ast.parse(source)

In [ ]:
tree = copy.deepcopy(original_tree)
tree.body[0].args.args[0].annotation = ast.Name('str')
tree.body[0].args.args[1].annotation = ast.Name('int')
with open("temp.py", "w") as file:
    file.write(ast.unparse(tree))

In [ ]:
from pytype.io import *

res = check_or_generate_pyi(options)

errorlog = res.context.errorlog
len(errorlog)

In [148]:
import builtins
import itertools
from pytype.io import *

builtin_types = [d for d in dir(builtins) if (isinstance(getattr(builtins, d), type)) and ord(d[0]) >= 97]
type_sample = ['bool',
 'bytearray',
 'bytes',
 'dict',
 'float',
 'int',
 'list',
 'map',
 'set',
 'slice',
 'staticmethod',
 'str',
 'tuple'
]

def test_types_function(original_tree, element):
    options = config.Options.create("temp.py")
    tree = copy.deepcopy(original_tree)
    function_def = tree.body[original_tree.body.index(element)]
    arguments = function_def.args.args
    possible_arg_types = list(itertools.permutations(type_sample, len(arguments)))
    tested_arg_types = []
    for arg_types in tqdm(possible_arg_types):
        for i, arg_type in enumerate(arg_types):
            arguments[i].annotation = ast.Name(arg_type)
        with open("temp.py", "w") as file:
            file.write(ast.unparse(tree))      
        res = check_or_generate_pyi(options)
        errorlog = res.context.errorlog
        if len(errorlog) == 0:
            tested_arg_types.append(arg_types)
    return tested_arg_types
tested_arg_types = test_types_function(original_tree, original_tree.body[0])

100%|██████████| 156/156 [00:19<00:00,  7.87it/s]


In [146]:
with open("temp.py", "w") as file:
    file.write(ast.unparse(original_tree)) 

In [151]:
import inspect
import importlib

sub_module = importlib.import_module("numpy._core.fromnumeric")
source = inspect.getsource(sub_module)
original_tree = ast.parse(source)